In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend
import tensorflow
tensorflow.set_random_seed(7)
from sklearn.model_selection import train_test_split
import numpy
numpy.random.seed(7)

Using TensorFlow backend.


## Restricting GPU memory usage

The code here should be added to any work you do on Volta.  If you don't, then your code will monopolize all available memory on each of the 4 GPUs on the machine, preventing others from working on it.  If you do **that**, you will be frowned upon.

The code in the next cell has the effect that:
1. Memory use will start off with some small fraction of the memory on each GPU.
1. It will grow if necessary (since `allow_growth` is set to `True`).
1. It will max out at 5% of overall memory.  Given the GPUs we have, this gives you (4 x 808 MB), which should be sufficient here.

In [2]:
########################
# Limit TensorFlow GPU use.
config = tensorflow.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.05
backend.tensorflow_backend.set_session(tensorflow.Session(config=config))
########################

## Loading our data

The function `train_test_split` comes from `sklearn` (from `SciKit`). Two things to note:
1. Setting `test_size` to `0.25` means that 25% of the data is held aside as testing data.  
1. Randomness here can affect overall results.

In [3]:
dataset = numpy.loadtxt("diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.25)

## Building and evaluating our model

The initial model is a simple network, with two hidden layers, each of which uses the RELU function for activation.  There is also a simple single-class output neuron using the Sigmoid logistic function.  The loss function is binary cross entropy (and so we want to measure the associated binary accuracy value).  The model is trained using a batch size equal to all of the training data, for 1,000 epochs.

For this network, do the following:
1. Binary cross entropy iss for multi-label classifications function is one of loss function to calculate the predicted probability diverges from the actual label.  ![image](  https://ml-cheatsheet.readthedocs.io/en/latest/_images/cross_entropy.png)       .then you sum all the labal's diverages from actual labal.The reason is this function is really good at classfication for actual output is binary. And we alrealdy use one hot encoding to change labal to binary labal. And i think the log loss increases rapidly when predicted probability approches 0,so that can make change more significantly.
1. When epochs is 1000,the accuracy of final model on training data is 82.99% and accuracy of final model on testing data is 71.88%. When epochs is 5000,the accurcy of final model on training data is 91.15% and accurcy of final model on testing data is 67.19%. It is clearly that when you increase epochs , the accuray of model on trainning data increases but the accurcy of model on trainning data decreases. The reason is overfitting, when you increase epochs, the neural network will learn more information from trainning data so it makes that doesn't generalize well from training data to unseen data.
1. I add two dropout layers, one after each of the 25-neuron Dense layers and both of them set to 0.2,then the accuracy of final model on training data is 68.92%,and the accurcy of final model on testing data is 65.10%.We can clearly find both of two accurcy decrease.The reason why it happens is this function make network can not get sufficient trainning .And i do expirement that change 0.2 to 0.9, and the accuracy of final model on training data is 58.16%,and the accurcy of final model on testing data is 55.21%. It is caused by dropping out too much information, and network can not train well. Then i try to set epochs to 10000 the accuracy of final model on training data is 65.62%,and the accurcy of final model on testing data is 65.34%. ,even throngh Dropout(0.9).Because more epochs make network can lean more information.

In [10]:
# create model
model = Sequential()
model.add(Dense(25, input_dim=8, activation='relu'))
model.add(Dropout(0.9))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.9))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# Fit the model
model.fit(train_x, train_y, epochs=10000, batch_size=len(train_x), validation_data=(test_x, test_y), verbose=1)

Train on 576 samples, validate on 192 samples
Epoch 1/10000
576/576 [==============================] - 1s 901us/step - loss: 3.3500 - binary_accuracy: 0.6128 - val_loss: 4.2385 - val_binary_accuracy: 0.5052
Epoch 2/10000
576/576 [==============================] - 0s 17us/step - loss: 3.9937 - binary_accuracy: 0.5868 - val_loss: 4.2058 - val_binary_accuracy: 0.5052
Epoch 3/10000
576/576 [==============================] - 0s 15us/step - loss: 3.5031 - binary_accuracy: 0.6128 - val_loss: 4.1612 - val_binary_accuracy: 0.5052
Epoch 4/10000
576/576 [==============================] - 0s 17us/step - loss: 3.7406 - binary_accuracy: 0.5851 - val_loss: 4.1043 - val_binary_accuracy: 0.5052
Epoch 5/10000
576/576 [==============================] - 0s 19us/step - loss: 3.6881 - binary_accuracy: 0.5955 - val_loss: 4.0567 - val_binary_accuracy: 0.5052
Epoch 6/10000
576/576 [==============================] - 0s 17us/step - loss: 3.8943 - binary_accuracy: 0.5694 - val_loss: 4.0119 - val_binary_accuracy: 

Epoch 52/10000
576/576 [==============================] - 0s 14us/step - loss: 3.6438 - binary_accuracy: 0.5868 - val_loss: 3.7262 - val_binary_accuracy: 0.6302
Epoch 53/10000
576/576 [==============================] - 0s 17us/step - loss: 2.7616 - binary_accuracy: 0.6128 - val_loss: 3.7540 - val_binary_accuracy: 0.6302
Epoch 54/10000
576/576 [==============================] - 0s 15us/step - loss: 3.1290 - binary_accuracy: 0.6198 - val_loss: 3.7824 - val_binary_accuracy: 0.6302
Epoch 55/10000
576/576 [==============================] - 0s 14us/step - loss: 2.8831 - binary_accuracy: 0.6163 - val_loss: 3.8030 - val_binary_accuracy: 0.6302
Epoch 56/10000
576/576 [==============================] - 0s 15us/step - loss: 3.2527 - binary_accuracy: 0.6024 - val_loss: 3.8289 - val_binary_accuracy: 0.6354
Epoch 57/10000
576/576 [==============================] - 0s 16us/step - loss: 3.2806 - binary_accuracy: 0.6146 - val_loss: 3.8556 - val_binary_accuracy: 0.6354
Epoch 58/10000
576/576 [==========

Epoch 103/10000
576/576 [==============================] - 0s 14us/step - loss: 3.0818 - binary_accuracy: 0.6128 - val_loss: 3.9805 - val_binary_accuracy: 0.6354
Epoch 104/10000
576/576 [==============================] - 0s 13us/step - loss: 2.5896 - binary_accuracy: 0.6441 - val_loss: 3.9727 - val_binary_accuracy: 0.6354
Epoch 105/10000
576/576 [==============================] - 0s 14us/step - loss: 3.0284 - binary_accuracy: 0.6111 - val_loss: 3.9495 - val_binary_accuracy: 0.6302
Epoch 106/10000
576/576 [==============================] - 0s 15us/step - loss: 3.0469 - binary_accuracy: 0.6059 - val_loss: 3.9337 - val_binary_accuracy: 0.6302
Epoch 107/10000
576/576 [==============================] - 0s 14us/step - loss: 2.9921 - binary_accuracy: 0.6337 - val_loss: 3.9211 - val_binary_accuracy: 0.6354
Epoch 108/10000
576/576 [==============================] - 0s 16us/step - loss: 2.8826 - binary_accuracy: 0.6163 - val_loss: 3.9165 - val_binary_accuracy: 0.6354
Epoch 109/10000
576/576 [===

Epoch 154/10000
576/576 [==============================] - 0s 18us/step - loss: 2.7184 - binary_accuracy: 0.6250 - val_loss: 4.3720 - val_binary_accuracy: 0.6354
Epoch 155/10000
576/576 [==============================] - 0s 13us/step - loss: 2.4838 - binary_accuracy: 0.6597 - val_loss: 4.3895 - val_binary_accuracy: 0.6354
Epoch 156/10000
576/576 [==============================] - 0s 13us/step - loss: 2.6064 - binary_accuracy: 0.6146 - val_loss: 4.3980 - val_binary_accuracy: 0.6354
Epoch 157/10000
576/576 [==============================] - 0s 14us/step - loss: 3.0310 - binary_accuracy: 0.6111 - val_loss: 4.4059 - val_binary_accuracy: 0.6354
Epoch 158/10000
576/576 [==============================] - 0s 13us/step - loss: 3.1279 - binary_accuracy: 0.6076 - val_loss: 4.4128 - val_binary_accuracy: 0.6354
Epoch 159/10000
576/576 [==============================] - 0s 15us/step - loss: 2.6706 - binary_accuracy: 0.6163 - val_loss: 4.4156 - val_binary_accuracy: 0.6354
Epoch 160/10000
576/576 [===

Epoch 205/10000
576/576 [==============================] - 0s 16us/step - loss: 2.5536 - binary_accuracy: 0.6302 - val_loss: 5.6000 - val_binary_accuracy: 0.6354
Epoch 206/10000
576/576 [==============================] - 0s 16us/step - loss: 2.8799 - binary_accuracy: 0.6128 - val_loss: 5.6103 - val_binary_accuracy: 0.6354
Epoch 207/10000
576/576 [==============================] - 0s 16us/step - loss: 2.4810 - binary_accuracy: 0.6285 - val_loss: 5.6186 - val_binary_accuracy: 0.6354
Epoch 208/10000
576/576 [==============================] - 0s 15us/step - loss: 2.5942 - binary_accuracy: 0.6302 - val_loss: 5.6266 - val_binary_accuracy: 0.6354
Epoch 209/10000
576/576 [==============================] - 0s 15us/step - loss: 2.7525 - binary_accuracy: 0.6111 - val_loss: 5.6336 - val_binary_accuracy: 0.6354
Epoch 210/10000
576/576 [==============================] - 0s 16us/step - loss: 2.6917 - binary_accuracy: 0.6372 - val_loss: 5.6403 - val_binary_accuracy: 0.6354
Epoch 211/10000
576/576 [===

...

576/576 [==============================] - 0s 13us/step - loss: 0.9674 - binary_accuracy: 0.6493 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9791/10000
576/576 [==============================] - 0s 14us/step - loss: 1.0945 - binary_accuracy: 0.6580 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9792/10000
576/576 [==============================] - 0s 14us/step - loss: 1.0986 - binary_accuracy: 0.6458 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9793/10000
576/576 [==============================] - 0s 13us/step - loss: 1.1531 - binary_accuracy: 0.6458 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9794/10000
576/576 [==============================] - 0s 18us/step - loss: 0.8855 - binary_accuracy: 0.6545 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9795/10000
576/576 [==============================] - 0s 16us/step - loss: 0.9609 - binary_accuracy: 0.6597 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9796/10000
576/576 [=============

Epoch 9841/10000
576/576 [==============================] - 0s 13us/step - loss: 0.9886 - binary_accuracy: 0.6562 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9842/10000
576/576 [==============================] - 0s 14us/step - loss: 0.9357 - binary_accuracy: 0.6545 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9843/10000
576/576 [==============================] - 0s 13us/step - loss: 1.0901 - binary_accuracy: 0.6615 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9844/10000
576/576 [==============================] - 0s 14us/step - loss: 1.0352 - binary_accuracy: 0.6615 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9845/10000
576/576 [==============================] - 0s 13us/step - loss: 0.9180 - binary_accuracy: 0.6510 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9846/10000
576/576 [==============================] - 0s 16us/step - loss: 0.9360 - binary_accuracy: 0.6528 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9847/10000
576/5

Epoch 9892/10000
576/576 [==============================] - 0s 13us/step - loss: 1.0105 - binary_accuracy: 0.6545 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9893/10000
576/576 [==============================] - 0s 14us/step - loss: 1.1798 - binary_accuracy: 0.6493 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9894/10000
576/576 [==============================] - 0s 13us/step - loss: 1.0747 - binary_accuracy: 0.6545 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9895/10000
576/576 [==============================] - 0s 14us/step - loss: 0.9878 - binary_accuracy: 0.6493 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9896/10000
576/576 [==============================] - 0s 14us/step - loss: 0.9851 - binary_accuracy: 0.6632 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9897/10000
576/576 [==============================] - 0s 14us/step - loss: 1.1220 - binary_accuracy: 0.6493 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9898/10000
576/5

Epoch 9943/10000
576/576 [==============================] - 0s 15us/step - loss: 0.9926 - binary_accuracy: 0.6528 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9944/10000
576/576 [==============================] - 0s 15us/step - loss: 1.0437 - binary_accuracy: 0.6493 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9945/10000
576/576 [==============================] - 0s 14us/step - loss: 0.8848 - binary_accuracy: 0.6562 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9946/10000
576/576 [==============================] - 0s 14us/step - loss: 0.9946 - binary_accuracy: 0.6580 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9947/10000
576/576 [==============================] - 0s 14us/step - loss: 1.1418 - binary_accuracy: 0.6545 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9948/10000
576/576 [==============================] - 0s 14us/step - loss: 0.8525 - binary_accuracy: 0.6545 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9949/10000
576/5

Epoch 9994/10000
576/576 [==============================] - 0s 16us/step - loss: 0.9669 - binary_accuracy: 0.6562 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9995/10000
576/576 [==============================] - 0s 15us/step - loss: 1.0709 - binary_accuracy: 0.6493 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9996/10000
576/576 [==============================] - 0s 13us/step - loss: 1.1705 - binary_accuracy: 0.6562 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9997/10000
576/576 [==============================] - 0s 13us/step - loss: 0.9946 - binary_accuracy: 0.6545 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9998/10000
576/576 [==============================] - 0s 15us/step - loss: 0.9902 - binary_accuracy: 0.6597 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 9999/10000
576/576 [==============================] - 0s 13us/step - loss: 0.8866 - binary_accuracy: 0.6545 - val_loss: 5.8733 - val_binary_accuracy: 0.6354
Epoch 10000/10000
576/

## Evaluation of final model on training data

In [11]:
# evaluate the model
scores = model.evaluate(train_x, train_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

576/576 [==============================] - 0s 55us/step

binary_accuracy: 65.62%


## Evaluation of final model on testing data

In [12]:
# evaluate the model
scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

192/192 [==============================] - 0s 69us/step

binary_accuracy: 63.54%
